In [1]:
import requests
from pymongo import GEOSPHERE
import statistics
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient

In [2]:
import sys
sys.path.append('../')
import src.functions as fn

In [3]:
client = MongoClient("mongodb://localhost/companies")
db=client.get_database("companies")
c = db.get_collection("companies")

In [4]:
proj ={"name":1, "offices.country_code":1,"offices.latitude":1,
       "offices.longitude":1,"_id":0}
cond1 ={"tag_list":{"$regex": "games"}}
cond2 ={"total_money_raised": {"$regex":"\d.M"}}
seeds = list(c.find({"$and":[cond1,cond2]}, proj).sort("total_money_raised",-1))

In [6]:
fn.clean(seeds)

[{'name': 'Bigpoint',
  'offices': [{'country_code': 'DEU',
    'latitude': 53.5973055,
    'longitude': 9.9762519}]},
 {'name': 'eRepublik',
  'offices': [{'country_code': 'ESP',
    'latitude': 40.43094,
    'longitude': -3.6952896}]},
 {'name': 'Apaja',
  'offices': [{'country_code': 'FIN',
    'latitude': 60.171821,
    'longitude': 24.945167}]},
 {'name': 'Zynga',
  'offices': [{'country_code': 'USA',
    'latitude': 37.765158,
    'longitude': -122.404234}]},
 {'name': 'WildTangent',
  'offices': [{'country_code': 'USA',
    'latitude': 37.09024,
    'longitude': -95.712891}]},
 {'name': 'Heyzap',
  'offices': [{'country_code': 'USA',
    'latitude': 37.7905538,
    'longitude': -122.4041485}]},
 {'name': 'Metacafe',
  'offices': [{'country_code': 'USA',
    'latitude': 37.437328,
    'longitude': -122.159928}]},
 {'name': 'Zango',
  'offices': [{'country_code': 'USA',
    'latitude': 47.579208,
    'longitude': -122.155668}]},
 {'name': 'Hands-On Mobile',
  'offices': [{'country

In [7]:
locations = [fn.type_point(office["longitude"],office["latitude"]) for seed in seeds for office in seed["offices"]]

In [8]:
locations[0]

{'type': 'Point', 'coordinates': [9.9762519, 53.5973055]}

In [9]:
client_id = os.getenv("Client_Id")
client_secret = os.getenv("Client_Secret")
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
map_name = ["venue", "name"]
map_lat = ["venue", "location", "lat"]
map_long = ["venue", "location", "lng"]
map_distance = ["venue", "location", "distance"]
map_direction = ["venue", "location", "formattedAddress"]

In [11]:
conditions = [
    ["design",5000,"m",5],
    ["school",8000,"m",4],
    ["Starbucks",1000,"s",7],
    ["airport",10000,"s",6],
    ["karaoke",2000,"m",5],
    ["vegan",5000,"s",7],
    ["basketball stadium",10000,"s",3],
    ["vet",6000,"s",8]
]

In [16]:
scoring=[]
for i,location in enumerate(locations):
    score = dict()
    for condition in conditions:
        parametros = {
            "client_id": client_id,
            "client_secret": client_secret,
            "radius": f"condition[1]",
            "v": "20180323",
            "ll": f"{location['coordinates'][1]}, {location['coordinates'][0]}",
            "query": f"{condition[0]}",
            "limit": 150}
        try:
            resp = requests.get(url_query, params=parametros).json()["response"]["groups"][0]["items"]
       

            newjson = []
            for dic in resp:
                forlist= {}
                forlist["name"]= fn.getFromDict(dic,map_name)
                forlist["lat"]= fn.getFromDict(dic,map_lat)
                forlist["long"]= fn.getFromDict(dic,map_long)
                forlist["distance"]= fn.getFromDict(dic,map_distance)
                forlist["direction"]= fn.getFromDict(dic,map_direction)
                forlist["location"]= fn.type_point(forlist["lat"],forlist["long"])
                newjson.append(forlist)
        
       
            distances = [item["distance"] for item in newjson]
        except:
            distances =[]
            print("resp fail")
        if len(distances) != 0:
            if condition[2]=="s":
                 score[condition[0]]= condition[3]*1/min(distances)
            else:
                 score[condition[0]]= condition[3]*1/statistics.mean(distances)
        else:
            score[condition[0]]= 0
    score["total"]= sum(list(score.values()))
    score["location"]=location
    scoring.append(score)

In [18]:
condition[2]

's'

In [17]:
sc = pd.DataFrame(scoring)
sc

,design,school,Starbucks,airport,karaoke,vegan,basketball stadium,vet,total,location
0,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [9.9762519, 5..."
1,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [-3.6952896, ..."
2,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [24.945167, 6..."
3,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [-122.404234,..."
4,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [-95.712891, ..."
5,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [-122.4041485..."
6,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [-122.159928,..."
7,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [-122.155668,..."
8,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [-122.419204,..."
9,0,0,0,0,0,0,0,0,0,"{'type': 'Point', 'coordinates': [-122.335943,..."
